In [ ]:
import os
os.chdir("../")

from Model.EmoClassifier import TextClassifierModule
from Data.Preprocessing import prepare_data
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
import torch
from Model.CAVInterpretationModule import CAVInterpretationModule
from Explanations.Concept import assemble_concepts_from_adjectives, load_concepts_from_file, enhance_concepts_with_synonyms, assemble_concepts_from_dataset
from Explanations.ConceptExplainer import ConceptExplainer
from Model.SingletonModelLoader import SingletonModelLoader
from Explanations.AttributionExplainer import AttributionExplainer
from Common.Concepts import CONCEPTS
from Model.CAVInterpretationModule import CAVInterpretationModule
from Model.SingletonModelLoader import SingletonModelLoader
from captum.attr import visualization as viz
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Setup hot realoading of modules when files in working dir change
%load_ext autoreload
%autoreload 2

In [5]:
combined_data_path = "./datasets/combined_data.csv"

In [9]:
data_module,classes = prepare_data(data_path=combined_data_path,
                                label_name="broadEmo",
                                dataset_names=["Empathetic Dialogues"], 
                                split_train_val_test=[0.8,0.1,0.1], 
                                batch_size=32, 
                                RANDOM_SEED=5)

Training data shape:  (19671, 4) 
Validation data shape:  (2459, 4) 
Test data shape:  (2459, 4)


## Concept Explanations with CAVs

In [10]:
# Load pre-stored adjectives for each emotion

adjectives = load_concepts_from_file('./concepts/CAV/concepts_adjectives.json')
concepts = assemble_concepts_from_adjectives(adjectives.keys(), adjectives)

In [11]:
explainer = ConceptExplainer('{}/model_checkpoints/emotion-recognizer-broad.ckpt'.format(os.getcwd()),
                             './dataset/cav_adjectives',
                             'classifier.2',
                             concepts
                             )

Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/captum/concept/_utils/classifier.py:131: UserWarning: Using default classifier for TCAV which keeps input both train and test datasets in the memory. Consider defining your own classifier that doesn't rely heavily on memory, fo

In [12]:
# Compute similarity with CAVs and return top similar CAVs with their emotion label

explainer.get_most_similar_concepts('Everyone got salary raise except for me!', 5)

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[Concept(2, 'Exasperation'),
 Concept(18, 'Disappointment'),
 Concept(16, 'Longing'),
 Concept(1, 'Envy'),
 Concept(0, 'Disgust')]

## Attribution Explanations

In [13]:
explainer = AttributionExplainer('{}/model_checkpoints/emotion-recognizer-fine.ckpt'.format(os.getcwd()))

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
# Compute Layer Integrated Gradients attributions for each word with respect to emotions with top
# predicted scores

result = explainer.interpret_emotion_prediction_top_k("I think we are not going to make it in time.", 3, n_steps=10)

In [15]:
print(viz.visualize_text(result['vizualization']))

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
fear,fear (0.44),fear,1.54,#s I think we are not going to make it in time . #/s
fear,terrified (0.22),terrified,1.25,#s I think we are not going to make it in time . #/s
fear,worry (0.08),worry,-1.57,#s I think we are not going to make it in time . #/s


<IPython.core.display.HTML object>


In [ ]:
# Run Layer Conductance Attributions

result = explainer.interpret_layer_attributions("I think we are not going to make it in time.")

In [ ]:
tokens, layer_attrs, emotion = result["tokens"], result["layer_attributions"], result["emotion"]

fig, ax = plt.subplots(figsize=(20,8))

print("Predicted Emotion: {}".format(emotion))

xticklabels=tokens
yticklabels=list(range(1,13))
ax = sns.heatmap(np.array(layer_attrs), xticklabels=xticklabels, yticklabels=yticklabels, linewidth=0.2, cmap="rocket")
plt.xlabel('Tokens')
plt.ylabel('Layers')
plt.show()